In [1]:
print("OK!")

OK!


In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install transformers
!pip install huggingface-hub
!pip install ctransformers
!pip install langchain-community
!pip install pypdf
!pip install sentence_transformers

In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [4]:
import os

api_key = os.environ.get("b5ffef4a-1b34-47e3-ad9d-95b79d9750c8") or 'b5ffef4a-1b34-47e3-ad9d-95b79d9750c8'
PINECONE_API_ENV = "us-east-1"

In [39]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [40]:
extracted_data = load_pdf("/content/deneme/")

In [41]:
extracted_data

[Document(metadata={'source': '/content/deneme/ornek_dataset.pdf', 'page': 0}, page_content='JSON Verileri\n{\n"Belgenin türü": "Fatura",\n"Fatura tarihi": "21-01-2032",\n"Fatura Numaras■": "0123456789101",\n"Fatura Kesilen": "Harun Dürüst\\n+0 212 123 24 25\\nAtatürk Mah. Gazi Bulvar■ No: 12\\n10100 Karesi/Bal■kesir",\n"Ürün Ad■": [\n"Kad■n Montü",\n"Yeni Sezon Bot",\n"Kad■n Aksesuar",\n"Kad■n Ayakkab■",\n"Modern Çanta"\n],\n"Ürün Adeti": [\n1,\n1,\n2,\n3,\n1\n],\n"Ürün Fiyat■": [\n"1000,00 TL",\n"1500,00 TL",\n"250,00 TL",\n"150,00 TL",\n"150,00 TL"\n],\n"Ürün Toplam Fiyat■": [\n"1000,00 TL",\n"1500,00 TL",\n"500,00 TL",\n"450,00 TL",\n"150,00 TL"\n],\n"Ara Toplam": "3600,00 TL",\n"Vergi (%15)": "540,00 TL",\n"Genel Toplam": "4140,00 TL",\n"Ödeme Bilgileri": [\n"Ödeme Bilgisi\\nAkkallar Bank\\nHesap Ad■: Akkallar Giyim ■irketi\\nHesap No: 123 456 789 00\\nÖdeme Vadesi: 01 Ocak 2026",\n"+0 212 123 24 25\\nwww.harikasite.web.tr\\nAtatürk Mah. Gazi Bulvar■ No: 12\\n10100 Karesi/Bal■kesi

In [42]:
#Create text chunks PDF için
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [43]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 2


In [44]:
text_chunks

[Document(metadata={'source': '/content/deneme/ornek_dataset.pdf', 'page': 0}, page_content='JSON Verileri\n{\n"Belgenin türü": "Fatura",\n"Fatura tarihi": "21-01-2032",\n"Fatura Numaras■": "0123456789101",\n"Fatura Kesilen": "Harun Dürüst\\n+0 212 123 24 25\\nAtatürk Mah. Gazi Bulvar■ No: 12\\n10100 Karesi/Bal■kesir",\n"Ürün Ad■": [\n"Kad■n Montü",\n"Yeni Sezon Bot",\n"Kad■n Aksesuar",\n"Kad■n Ayakkab■",\n"Modern Çanta"\n],\n"Ürün Adeti": [\n1,\n1,\n2,\n3,\n1\n],\n"Ürün Fiyat■": [\n"1000,00 TL",\n"1500,00 TL",\n"250,00 TL",\n"150,00 TL",\n"150,00 TL"\n],\n"Ürün Toplam Fiyat■": [\n"1000,00 TL",\n"1500,00 TL",'),
 Document(metadata={'source': '/content/deneme/ornek_dataset.pdf', 'page': 0}, page_content='"1500,00 TL",\n"500,00 TL",\n"450,00 TL",\n"150,00 TL"\n],\n"Ara Toplam": "3600,00 TL",\n"Vergi (%15)": "540,00 TL",\n"Genel Toplam": "4140,00 TL",\n"Ödeme Bilgileri": [\n"Ödeme Bilgisi\\nAkkallar Bank\\nHesap Ad■: Akkallar Giyim ■irketi\\nHesap No: 123 456 789 00\\nÖdeme Vadesi: 01 Oca

In [45]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    return embeddings

In [46]:
embeddings = download_hugging_face_embeddings()

In [47]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
query_result

In [ ]:
!pip install langchain_pinecone

In [28]:
!pip install uuid

  Preparing metadata (setup.py) ... done
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6478 sha256=56373c79bd034b68bca997fe218f4e6de6c74482b5bbb572521a9ca564a04113
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [48]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4

PINECONE_API_KEY = "b5ffef4a-1b34-47e3-ad9d-95b79d9750c8"

# Initialize the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name = "json"
index = pc.Index(index_name)

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

uuids = [str(uuid4()) for _ in range(len(text_chunks))]

vectorstore.add_documents(documents=text_chunks, ids=uuids)

['b8ac03f6-5691-4d6b-a22e-38820918288b',
 'da29283b-9e4d-4c86-87c7-12bba5097aec']

In [49]:
#If we already have an index we can load it like this
#docsearch=vectorstore.from_existing_index(index_name, embeddings)

query = "KDV oranı kaçtır?"

docs=vectorstore.similarity_search(query, k=3)

print("Result", docs)

Result [Document(metadata={'page': 0.0, 'source': '/content/deneme/ornek_dataset.pdf'}, page_content='"1500,00 TL",\n"500,00 TL",\n"450,00 TL",\n"150,00 TL"\n],\n"Ara Toplam": "3600,00 TL",\n"Vergi (%15)": "540,00 TL",\n"Genel Toplam": "4140,00 TL",\n"Ödeme Bilgileri": [\n"Ödeme Bilgisi\\nAkkallar Bank\\nHesap Ad■: Akkallar Giyim ■irketi\\nHesap No: 123 456 789 00\\nÖdeme Vadesi: 01 Ocak 2026",\n"+0 212 123 24 25\\nwww.harikasite.web.tr\\nAtatürk Mah. Gazi Bulvar■ No: 12\\n10100 Karesi/Bal■kesir"\n]\n}'), Document(metadata={'page': 0.0, 'source': '/content/deneme/ornek_dataset.pdf'}, page_content="['1000,00 TL', '1500,00 TL', '500,00 TL', '450,00 TL', '150,00 TL']\n3600,00 TL\n540,00 TL\n4140,00 TL\n['Ödeme Bilgisi\\nAkkallar Bank\\nHesap Ad■: Akkallar Giyim ■irketi\\nHesap No: 123 456 789 00\\nÖdeme Vadesi: 01 Ocak 2026', '+0 212 123 24 25\\nwww.harikasite.web.tr\\nAtatürk Mah. Gazi Bulvar■ No: 12\\n10100 Karesi/Bal■kesir']"), Document(metadata={'page': 0.0, 'source': '/content/deneme

In [50]:
prompt_template="""
Kullanıcının sorusunu yanıtlamak için aşağıdaki bilgileri kullanın.
Soru Türkçe olacaktır ve siz de Türkçe cevap vermelisiniz.
Eğer cevabı bilmiyorsanız, sadece bilmediğinizi söyleyin, bir cevap uydurmaya çalışmayın.

İçerik: {context}
Soru: {question}

Sadece aşağıdaki yardımcı yanıtı gönderin, başka bir şey göndermeyin.
Yararlı cevap:
"""

In [51]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [52]:
llm=CTransformers(model="mradermacher/Trendyol-LLM-7b-chat-v1.8-GGUF",
                  model_file="Trendyol-LLM-7b-chat-v1.8.Q4_K_M.gguf",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained("mradermacher/Trendyol-LLM-7b-chat-v1.8-GGUF", model_file="Trendyol-LLM-7b-chat-v1.8.Q4_K_M.gguf", model_type="llama")

In [53]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
import time

while True:
    user_input = input("Input Prompt: ")

    start_time = time.time()  # Başlangıç zamanı
    result = qa({"query": user_input})
    end_time = time.time()    # Bitiş zamanı

    elapsed_time = end_time - start_time  # Geçen süre

    print("Response : ", result["result"])
    print(f"Time taken: {elapsed_time:.2f} seconds")


In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

Input Prompt:Maskotlar sahay ne zaman terk etmelidir?
Response :  Lütfen söyleyebilir misiniz hangi kulüpten bahsettiğinizi?


Input Prompt:Maskotlar müsabaka öncesinde sahayı ne zaman terk etmelidir?
Response :  Müsabaka başlamadan beş dakika önce maskotlar sahadan çıkmalı.
Input Prompt:Ev sahibi kulüpler, misafir seyirciler için en fazla ne kadar yer ayırabilirler?
Response :  Ev sahibi kulüp, misafir kulübün yazılı onayı ve İl veya İlçe Spor Güvenlik Kurulu'nun kararı ile misafir kulüp seyircileri için en fazla % 30 oranında yer ayırabilir.
Input Prompt:şampiyon olan kulübe kaç adet madalya teslim edilir?
Response :  
Şampiyon olan kulübe 35 adet madalya teslim ediliyor.
Input Prompt:Takımlar kaç adet yedek bayrağı bulundurmalıdır?
Response :  Takımların en az 4 adet yedek bayrağı bulundurması gerekir.


KeyboardInterrupt: Interrupted by user